# Scrape world GDP and GDP based on purchase parity (PPP)

World GDP and purchase parity adjusted GDP from 2015 were scraped from [Knoema](http://knoema.com).

In [14]:
import pandas as pd
import numpy as np
import requests

import re
import time
from StringIO import StringIO

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

from csv_pkl_sql import save_it

## Scrape the GDP tables

In [26]:
driver = webdriver.Chrome()

In [49]:
driver.get('https://knoema.com/nwnfkne/world-gdp-ranking-2015-data-and-charts')
html = driver.page_source

In [54]:
# For some reason, scraping this page only works when 
# I save it to a file first and then read it back
with open('gdp.html', 'w') as fh:
    fh.write(html.encode('utf-8'))
    
with open('gdp.html','r') as fh:
    html = fh.read()
    
tables = pd.read_html(html)

In [58]:
! rm -rf gdp.html gdp_files

In [62]:
gdp = tables[2].rename(columns={'Unnamed: 0':'country','Units':'units'})
gdp.head(1)

,country,units,2013,2014,2015
0,United States,U.S. dollars (Billions),16663.2,17348.1,17947.0


In [63]:
gdp_ppp = tables[3].rename(columns={'Unnamed: 0':'country','Units':'units'})
gdp_ppp.head(1)

,country,units,2013,2014,2015
0,China,Current international dollar (Billions),16468.2,17960.7,19392.4


Select just the countries of interest.

In [64]:
locations = pd.read_pickle('../pkl/00_cleaned_city_names.pkl')
locations.head(1)

,location,location_type,country,province,county,city
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,None,None


In [68]:
mask = gdp.country.isin(locations.country.unique())
gdp = gdp.loc[mask]
gdp.shape

(12, 5)

In [69]:
mask = gdp_ppp.country.isin(locations.country.unique())
gdp_ppp = gdp_ppp.loc[mask]
gdp_ppp.shape

(12, 5)

In [71]:
gdp_ppp.dtypes

country     object
units       object
2013       float64
2014       float64
2015       float64
dtype: object

In [72]:
gdp.dtypes

country     object
units       object
2013       float64
2014       float64
2015       float64
dtype: object

In [74]:
save_it(gdp, '09_GDP_table')

In [75]:
save_it(gdp_ppp, '09_GDP_PPP_table')